In [1]:
from cooking_bot.data_formats import *
from cooking_bot.encoders import get_clip_text_embedding, get_image_embedding, get_sentence_embedding, CLIP_MODEL, SENTENCE_TRANSORMER
from cooking_bot import REPO_PATH
from glob import glob
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import torch
import bertviz
from IPython.display import display
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModel
import os
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
os.chdir(REPO_PATH)

/home/felix/Documents/web/project


In [6]:
recipes = []

for id, path in enumerate(tqdm(glob("jsons/*.json"), "Filling Index")):
    
    with open(path, "r") as f:

        data = f.read()
    
    recipe = Recipe.model_validate_json(data)
    recipes.append(recipe)

Filling Index: 100%|██████████| 994/994 [00:31<00:00, 31.09it/s] 


In [7]:
many_instructions = [i for i in recipes if len(i.instructions) > 6]

def number_of_images(x : Recipe):
    
    s = 0
    
    for i in x.instructions:
        for img in i.stepImages:
            s += img.embedding is not None
            
    return s




many_instructions.sort(key=number_of_images, reverse=True)

len(many_instructions)

291

In [10]:
recipe : Recipe = many_instructions[6]



number_of_images(recipe), len(recipe.instructions)

(11, 12)

In [11]:
del recipes
    
del many_instructions

In [12]:
model_path = "sentence-transformers/all-MiniLM-L6-v2"
sentence_tokenizer = AutoTokenizer.from_pretrained(model_path)
sentence_config = AutoConfig.from_pretrained(model_path,  output_hidden_states=True, output_attentions=True, )  
sentence_model = AutoModel.from_pretrained(model_path, config=sentence_config)

In [39]:
english = [i.stepText for i in recipe.instructions][:9]
sentence = english[-4]
print(sentence)
sentence_tokens = sentence_tokenizer(sentence, return_tensors='pt', add_special_tokens=False, max_length = 512, padding=True, truncation = True )
with torch.no_grad():
    sentence_outputs = sentence_model(**sentence_tokens)

len(sentence_outputs["attentions"])

With a paper towel, pat dry the mushrooms and make sure the kale is very dry as well.


6

In [47]:
tokens = sentence_tokenizer.convert_ids_to_tokens(sentence_tokens["input_ids"][0]) 
attentions = sentence_outputs["attentions"]


bertviz.head_view(attentions, tokens, layer = 0, heads =(1,))


<IPython.core.display.Javascript object>